In [ ]:
# Mordred
from mordred import Calculator, descriptors

# Python standard library
import os, sys

# Data
import pandas as pd
import numpy as np
import seaborn as sns

# Plotting
import matplotlib.pyplot as plt
from PIL import Image

# Machine learning
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Utils
sys.path.append(".")
from utils import classification_metrics as cmetrics
from utils import finger_prints as fp
from utils import classification_workflow_functions as cwf

import logging 
logging.basicConfig(format='%(message)s')
log = logging.getLogger()
log.setLevel(logging.ERROR)

from dask.distributed import Client
try:
    client.shutdown()
except NameError:
    log.info("No client already active")

client = Client(dashboard_address=":8855")
log.info("Dask clinet on localhost:8855")

random_seed = 10459
np.random.seed = random_seed
np.random.RandomState(random_seed)
log.info(f"Random seed fixed as {random_seed} current working dir {os.getcwd()}")

# Prepare data

## Load

In [ ]:
os.makedirs("results_mordred", exist_ok=True)
os.chdir("results_mordred")

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [ ]:
data = pd.read_csv("../data/ccs-98.csv", sep=";")
data.columns = ["_".join(ent.lower().strip().split(" ")) for ent in data.columns]

In [ ]:
smiles = data["smiles"]
log.info("SMILES: {}".format(smiles.head()))

names = data["label"]
log.info("Names: {}".format(names.head()))

number_of_n_atoms = data["n_nitrogen"].astype("int")
log.info("Number of N atoms: {}".format(number_of_n_atoms.head()))

amines_mass_mr = data["molecular_mass"].astype("float64")
pd.to_numeric(amines_mass_mr, errors="coerce")
log.info("Amines mass Mr: {}".format(amines_mass_mr.head()))

molco2_moln = data["capacity_molco2_molamime"]
pd.to_numeric(molco2_moln, errors="coerce")
log.info("molCO2_molN: {}".format(molco2_moln.head()))

initial_rates = data["rate_molco2_molamime_min"]
pd.to_numeric(initial_rates, errors="coerce")
log.info("initial_rates: {}".format(initial_rates.head()))

## Set Target Properties

In [ ]:
targets = molco2_moln
target_name = "capacity (molCO2 / molN)"
target_key = "capacity_molco2_molamime"
units = "molco2_moln"
threshold_for_catagorical = 50.0

## Make Mordred Features

In [ ]:
calc = Calculator(descriptors, ignore_3D=False)
molecule_list = [cwf.get_mol_from_smiles(s) for s in smiles]
features_df = calc.pandas(molecule_list)

In [ ]:
features_df.dropna(inplace=True, thresh=int(0.9*len(features_df.index)))
threshold = 0.5
features_df.drop(features_df.std()[features_df.std() < threshold].index.values, axis=1)
features_df.columns = [ent.strip() for ent in features_df.columns]
features_df.head()

In [ ]:
log.info(features_df)
feature_types = "no_catagorical"

In [ ]:
features_df.to_csv("mordred.csv")

## Calculate significant featuers

In [ ]:
reasonable_predicted_properties, significant_fearures = cwf.find_correlating_features(features_df, targets, thresh=0.5, 
                                                                                      plot=False, corr_method="spearman", 
                                                                                      sig_metric="spearman", process_non_numeric=True, 
                                                                                      sig_level=0.05, significance=True, n_sample=5000)

In [ ]:
log.info("{} {}".format(reasonable_predicted_properties, len(reasonable_predicted_properties)))
log.info("{} {}".format(significant_fearures, len(significant_fearures)))
use_significant = True
use_reasonable = False

In [ ]:
feats_df = pd.DataFrame()

if use_significant is True:
    for k in significant_fearures:
        feats_df[k] = features_df[k]
        
elif use_reasonable is True:
    for k in reasonable_predicted_properties:
        feats_df[k] = features_df[k]
        
feats_df.to_csv("mordred-features.csv", index=False)

In [ ]:
feats_df

## Calculate classes

In [ ]:
counts = []
for ith, s in enumerate(smiles):
    n_primary, n_secondary, n_tertiary, n_aromaticsp2 = cwf.count_amine_types(s, show=False)
    counts.append([n_primary, n_secondary, n_tertiary, n_aromaticsp2])
    log.debug("\n{}; number of primary: {} number of secondary: {} number of tertiary: {} number of aromatic sp2 nitrogen atoms: {}\nsmiles {}". format(ith, n_primary, n_secondary, n_tertiary, n_aromaticsp2, s))

df = pd.DataFrame(data=counts, columns=["primary_amine_counts","secondary_amine_counts", "tertiary_amine_counts", "aromatic_sp2_n" ])
df.corr()

In [ ]:
if target_name == "initial_rate":
    log.info("Initial rate class")
    mean = np.mean(initial_rates)
    stdev = np.std(initial_rates)
    class_thresh = mean + stdev
    log.info("mean {} standard deviation {} class threshold {}".format(mean, stdev, class_thresh))
    classes = []
    for i in initial_rates:
        if i < class_thresh:
            classes.append(0)
        else:
            classes.append(1)
    log.info("Number of classes: {}  Number of class 1: {} number of class 0: {}".format(len(classes), len([x for x in classes if x == 1]), len([x for x in classes if x == 0])))
    class_targets_df = pd.DataFrame(np.array([classes]).T, columns=["classes"])
    features_and_classes_df = features_df.copy()
    features_and_classes_df["classes"] = classes
    
else:
    log.info("Capture capacity class")
    classes = cwf.capacity_classes(df["primary_amine_counts"].values, df["secondary_amine_counts"].values, df["tertiary_amine_counts"].values, df["aromatic_sp2_n"].values, targets,
                         units=units, number_N_atoms=number_of_n_atoms, amines_mr=amines_mass_mr)
    log.info(classes)
    log.info("Number of classes: {}  Number of class 1: {} number of class 0: {}".format(len(classes), len([x for x in classes if x == 1]), len([x for x in classes if x == 0])))

    class_targets_df = pd.DataFrame(np.array([classes]).T, columns=["classes"])
    features_and_classes_df = features_df.copy()
    features_and_classes_df["classes"] = classes

In [ ]:
threshold_for_catagorical = 50.0
log.info("Threshold for a catagorical feature is any feature where more than {}% of the point have have a value or all vlaues are separated by the same step size".format(threshold_for_catagorical))
catagorical_name = []
catagorical_indx = []
for ith, column in enumerate(feats_df.columns):
    if column != "training":
        vals = sorted(set(feats_df[column].values))
        steps = [elt - eltp1 for elt, eltp1 in zip(vals, vals[1:])]
        log.debug(f"\n{ith} {column}\n{steps}\n")
        percent = []
        for step in steps:
            percent.append(len([elt for elt in steps if elt == step])/len(steps) * 100.0)
        log.debug(f"percentage {ith} {column}: {percent}\n")

        if any(elt >= threshold_for_catagorical for elt in percent):
            log.info("More than {} point have the same value for {} {}".format(threshold_for_catagorical, ith, column))
            catagorical_indx.append(ith)
            catagorical_name.append(column)
        elif all(elt == steps[0] for elt in steps):
            log.info("Same separating step size for all features in {} {}".format(ith, column))

log.info(catagorical_indx)
log.info(catagorical_name)


## Data scaling

In [ ]:
feats = feats_df.columns
mordred_features_df = pd.DataFrame(data=np.array([[i+1 for i in range(len(feats))], feats]).T, columns=["Index", "Feature"])
with open("modred_{}_feature.tex".format(len(feats)), "w") as fout:
    mordred_features_df.to_latex(fout, float_format="{:0.2f}".format, position="H", longtable=True, caption="Feature selected from Spearman correlation coefficient (\textgreater 0.4) and two tail p test at 95\%", label="tbl:fingerprint_{}_features".format(len(feats)), index=False)

In [ ]:
feats_df_bkup = feats_df.copy()

In [ ]:
for inx, f in enumerate(feats):
    log.info(f"-----\nIndex: {inx}\n{feats_df[f].values}\n-----\n")

In [ ]:
feature_types = "some_catagorical"
# NOTE: USER DEFINES THE LIST BELOW!!!!!!
catagorical_indxs = [18, 19, 20, 21, 22, 28]
# [0, 1, 2, 5, 6, 7, 8, 9, 10,14, 15, 20, 31, 32, 33, 34]
# 0.6 [0, 1, 2, 3, 4]
# 0.5 [0, 1, 2, 5, 6, 7, 8, 9, 10,14, 15, 20, 31, 32, 33, 34]
# 0.4 [0, 1, 2, 25, 26, 27, 29, 30, 31, 32, 39, 40, 41, 47, 72, 73, 74, 75]
# 0.4 old [0, 1, 2, 21, 22, 23, 25, 26, 35, 36, 37, 41, 43, 68, 69, 70, 71]
feature_columns = feats_df.columns

# Backup
backup_feats_df = feats_df.copy()

# None catagorical only scale the data as numbers
if feature_types == "no_catagorical":
    mm_scaler = MinMaxScaler()
    feats_df = mm_scaler.fit_transform(feats_df)
    log.info(pd.DataFrame(feats_df, columns=feature_columns))
    feats_df = pd.DataFrame(feats_df, columns=feature_columns)
    
# Some catagorical - Need to provide the indexes
elif feature_types == "some_catagorical":
    numeric_features = [feature_columns[i] for i in range(len(feature_columns)) if i not in catagorical_indxs]
    numerical_transformer = MinMaxScaler()
    categorical_features = [feature_columns[i] for i in range(len(feature_columns)) if i in catagorical_indxs]
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    if any(ent in categorical_features for ent in numeric_features):
        log.warning("WARNING - numeric and catagorical feature specififed overlap")
        log.info(numeric_features)
        log.info(categorical_features)
    else:
        log.info("Numerical features:\n{} {}".format(numeric_features, len(numeric_features)))
        log.info("Catagorical features:\n{} {}".format(categorical_features, len(catagorical_indxs)))
    
    preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", numerical_transformer, numeric_features),
        ('catagorical', categorical_transformer, categorical_features)])
    
    feats_df = preprocessor.fit_transform(feats_df)
    feature_names = cwf.get_feature_names_from_column_transformers(preprocessor)
    catagorical_indxs = [i for i in range(len(numeric_features), len(feature_names))]
    log.info(feature_names)

    log.info(type(feats_df))
    try:
        log.info(pd.DataFrame(feats_df, columns=feature_names))
        feats_df = pd.DataFrame(feats_df, columns=feature_names)
    except ValueError:
        log.info(pd.DataFrame(feats_df.to_array(), columns=feature_names))
        feats_df = pd.DataFrame(feats_df.to_array(), columns=feature_names)
    log.info("catagorical indexes {}".format(catagorical_indxs))
    log.info("Catagorical features start on column name {} and end on {}".format(feats_df.columns[catagorical_indxs[0]], feats_df.columns[catagorical_indxs[-1]]))
    
# All catagorical
elif feature_types == "catagorical":
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    feats_df = categorical_transformer.fit_transform(feats_df).toarray()
    feature_names = [categorical_transformer.get_feature_names(feature_columns)]
    feats_df = pd.DataFrame(feats_df, columns=feature_names)
    log.info(feats_df)

# No scaling or other encoding
else:
    log.info("No scaling")

In [ ]:
continous_indexes = [ith for ith in range(0, catagorical_indxs[0])]

In [ ]:
feats_df.to_csv("mordred_scaled_onehotencode_features.csv", index=False)

# Train models

In [ ]:
classes_df = pd.DataFrame(data=classes, columns=["classes"])
log.info("Number in class 0: {}\nNumber in class 1: {}\nNumber of examples: {}".format(
    len([ith for ith in classes if ith == 0]), 
    len([ith for ith in classes if ith == 1]), len(classes)))

In [ ]:
kfold_clf_names = ["AdaBoost","Logistic Regression", "Gaussian Process"]

kfold_classifiers = [
    AdaBoostClassifier(random_state=random_seed),
    LogisticRegression(random_state=random_seed, n_jobs=-1, solver="lbfgs"),
    GaussianProcessClassifier(random_state=random_seed, n_jobs=-1)
]

kfold_classifier_parameters = {
    "AdaBoost": {"n_estimators": [10, 20, 50, 100]},
    "Logistic Regression": {"penalty":["l2", "none"], "C": [0.05, 0.1, 0.25, 0.5, 1.0, 1.25]},
    "Gaussian Process": {"kernel":[1.0 * Matern(length_scale=1.0, nu=1.5), 1.0 * Matern(length_scale=1.0, nu=2.5), 1.0 * RBF(1.0),  1.0 * RBF(1.0) + WhiteKernel(noise_level=0.5)]},
}

In [ ]:
pd.set_option('display.max_rows', 98)

In [ ]:
for ith, ent in enumerate(feats_df.isnull().sum().values):
    if ent != 0:
        log.info(f"Row {ith} is not free of nulls")

In [ ]:
cwf.kfold_test_imbalenced_classifiers_with_optimization(feats_df, classes_df, kfold_classifiers, kfold_classifier_parameters, 
                                                        overwrite=True, scale=False, cv=10, n_repeats=5, smiles=smiles, names=names,
                                                        random_seed=random_seed, clf_names=kfold_clf_names, class_labels=(0,1),
                                                        smote=True, smote_catagorical_indexes=catagorical_indxs, 
                                                        smote_continuous_indexes=continous_indexes)

In [ ]:
kfold_clf_names = ["AdaBoost","Logistic Regression", "Gaussian Process"]
directory_names = cwf.directory_names_from_classfier_names(kfold_clf_names)

# Analyse models

## Confusion matrix

In [ ]:
files_plt = []

for ith, dirname in enumerate(directory_names):
    log.info("\n{}\n-------------\n".format(dirname))
    data = cwf.build_data_from_directory(dirname, max_folds=5)
    
    log.debug("Last value in the data frame: {}".format(data[165:166]))
    
    probs = data[["prob0", "prob1"]].to_numpy()
    log.debug("Probablities for a few examples {}".format(probs[0:3,0:2]))
    
    cm = cmetrics.get_confusion_matrix(data, predicted_column_name="prediction", known_column_name="known", return_dict=False)
    log.debug("Confusion matrix for {}\n{}".format(dirname, cm))
    
    plt_name = "capacity_{}_mordred.png".format("_".join([ent.lower() for ent in dirname.split()]))
    files_plt.append(plt_name)
    log.info("Saving plot to {}\n{}".format(plt_name, files_plt))
    metrics = cmetrics.calculate_confusion_based_metrics(df=data, probabilities=probs, col_map="hsv", positive_label=1, 
                                                         plt_filename=plt_name, all_classes=False, get_roc_curve=True, 
                                                         get_pr_curve=False, annotate=True, vmin=0, vmax=85,
                                                         title="{}".format(kfold_clf_names[ith]))
    #log.info("{}".format("\n".join(["{}: {}".format(k, v) for k, v in metrics.items()])))
    
    metrics_for_paper = {
    "accuracy": metrics["accuracy"],
    "sensitivity": metrics["tpr"],
    "specificity": metrics["tnr"],
    "mcc": metrics["matthews_correlation_coefficient"],
    "precision": metrics["precision"],
    "g-mean": metrics["g-mean"]
    }
    
    if ith == 0:
        df_metrics_for_paper = pd.DataFrame(data=metrics_for_paper, index=[kfold_clf_names[ith].lower()])
    else:
        df_metrics_for_paper = df_metrics_for_paper.append(pd.Series(metrics_for_paper, name=kfold_clf_names[ith].lower()))
    log.debug(df_metrics_for_paper)

with open("capacity_metrics_mordred.tex", "w") as fout:
    cap = "Classifier metrics for balanced data for capacity with models built from mordred features. MCC is the Matthew’s correlation coefficent."
    df_metrics_for_paper.to_latex(fout, float_format="{:0.2f}".format, position="H", caption=cap, label="tbl:mordred_features")
log.info(df_metrics_for_paper.to_latex())

## Feature importance

In [ ]:
fimp = pd.read_csv("importance_lr.csv")

In [ ]:
fimp

In [ ]:
fimp_mean = fimp.mean(axis=0)
means = fimp_mean.values
fimp_means = pd.DataFrame(means).transpose()
fimp_means.columns=feats_df.columns

fimp_sigma = fimp.std(axis=0)
sigmas = fimp_sigma.values
fimp_sigmas = pd.DataFrame(sigmas).transpose()
fimp_sigmas.columns=feats_df.columns

In [ ]:
max([len(" ".join(ent.split("_"))) for ent in fimp_means.columns])

In [ ]:
figure = plt.figure(figsize=(25,20))
plt.bar(x=[" ".join(ent.split("_")) for ent in fimp_means.columns], 
        height=fimp_means.iloc[0,:].values, 
        width=1.0,
        edgecolor="k",
        align="edge")
bins = np.arange(len(fimp_means.columns))
plt.xlim([0,bins.size])
plt.xlabel("Feature", fontsize=35)
plt.ylabel("Mean Coefficent value", fontsize=35)
plt.xticks(rotation=90, fontsize=20)
plt.yticks(fontsize=20)
plt.title("Logistic Regression Mean Feature Importance using Mordred fingerprints", fontsize=35)
plt.grid(True)
plt.tight_layout()
plt.savefig("feature_importance_lr_mordred.png")